In [1]:
import re

In [2]:
with open("Green Book.srt", "r") as f:
    lines = f.readlines()
    lines = [line.strip() for line in lines]
segments = []
for line in lines:
    if '-->' in line:
        time_matches = re.findall(r'\d{2}:\d{2}:\d{2},\d{3}', line)
        if len(time_matches) == 2:
            h1, m1, s1 = map(float, time_matches[0].replace(',', '.').split(':'))
            start_time = h1 * 3600.0 + m1 * 60.0 + s1 - 1.50
            h2, m2, s2 = map(float, time_matches[1].replace(',', '.').split(':'))
            end_time = h2 * 3600.0 + m2 * 60.0 + s2 - 1.50
            segments.append((start_time, end_time))

In [3]:
# load greenbook_01.wav to greenbook_19.wav and concatenate them into numpy array
import soundfile as sf
import numpy as np

filenames = ["green_book/greenbook_{:02d}.wav".format(i) for i in range(1, 20)]
concatenated_audio = []

for filename in filenames:
    audio, samplerate = sf.read(filename)
    concatenated_audio.append(audio)

concatenated_audio = np.concatenate(concatenated_audio)


In [4]:
audio_segments = []
for segment in segments:
    start_time, end_time = segment
    start_index = int(start_time * samplerate)
    end_index = int(end_time * samplerate)
    audio_segment = concatenated_audio[start_index:end_index]
    audio_segments.append(audio_segment)    

In [5]:
seeg_contacts = np.load("seeg_contacts.npy")
seeg_sr = 1024.0
seeg_segments = []
for segment in segments:
    start_time, end_time = segment
    start_index = int(start_time * seeg_sr)
    end_index = int(end_time * seeg_sr)
    seeg_segment = seeg_contacts[:, start_index:end_index]
    seeg_segments.append(seeg_segment)


In [6]:
np.save('data_segmented.npy', {'audio': audio_segments, 'seeg': seeg_segments})